# Introducción

Se ha notado que hay ciertos profesores que no consiguen acercar las adaptaciones necesarias a sus alumnos. El objetivo de nuestra tecnología sería predecir el nivel académico que va a desempeñar el alumno para estar más pendientes de aquellos alumnos que lo necesiten.

Para ello, se va a utilizar un dataset que contiene información sobre los alumnos y su rendimiento académico. Se va a realizar un análisis exploratorio de los datos para entender mejor la información que contienen y poder realizar un modelo predictivo que nos permita predecir el nivel académico de los alumnos.

# Dataset

## Columnas del dataset que vamos a usar
- `Gender`: Género del alumno
- `Age` : Edad del alumno
- `Study Hour/Day` : Horas de estudio al día
- `Learning mode` : Modalidad de aprendizaje
- `How many hour do you spent daily in social media?` : Horas diarias en redes sociales
- `Average attendance` : Asistencia media
- `With whom you are living?` : Con quién vive
- `What was your previous SGPA?` : Nota media anterior
- `What is your monthly family income?` : Nota media anterior

Lo primero que hemos hecho ha sido renombrar algunas de las columnas ya que no estaban muy bien redactadas. Además, hemos eliminado las columnas que no vamos a utilizar en nuestro análisis. 

Lo siguiente que hemos hecho ha sido eliminar las filas que contienen valores nulos.

Siguiendo esto hemos normalizado datos que estaban en formato de texto a formato numérico.

## Estandarizacion de tiempos

Lo que hemos hecho en el codigo que sigue este bloque de markdown es estandarizar los tiempos para poder tener los datos correctos tanto como para el Study Time como para el Time on Social Media.

In [ ]:
#Limpieza del dataset de los datos de los estudiantes.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import regex as re

#Cargamos el dataset

data = pd.read_csv('Students_Performance_data_set.csv')

pd.set_option('display.max_columns', None)

data.shape

#Renombramos las columnas que nos interesan

new_columns = {'Gender':'Gender', 'Age':'Age', 'Study Hour/day':'Study time', 'Learning mode':'Learning mode', 'How many hour do you spent daily in social media?':'Time on social media', 'Average attendance on class':'Average attendance', 'With whom you are living with?':'Cohabitants', 'What was your previous SGPA?':'Previous SGPA', 'What is your current CGPA?':'Current CPGA', 'What is your monthly family income?':'Family income'}

data.rename(columns=new_columns, inplace=True)

#Seleccionamos las columnas que nos interesan

columns_of_interest = ['Gender', 'Age', 'Study time', 'Learning mode', 'Time on social media', 'Average attendance', 'Cohabitants', 'Previous SGPA', 'Current CPGA',  'Family income']

data = data[columns_of_interest]

#Eliminamos las filas con valores nulos
data = data.dropna()


#Estandarizamos los valores de distintas columnas

## Study time ------------------------------------------------------------------------------------------------------

### Usamos la función unique para ver los valores únicos de la columna y asi saber como estandarizarlos
# print(data['Study time'].unique())

def standarise_time(time): # sigue teniendo un error en la función cuando tiene un valor de minutos y otro seguido de horas
    # we extract the number and hour/minute from the string
    hours_numeric = re.findall(r'\d+', str(time))
    
    # extract the numeric value for the hours
    if 'hours' in str(time).lower() or 'hour' in str(time).lower() or 'hrs' in str(time).lower() or 'hr' in str(time).lower():
        if hours_numeric:
            return int(hours_numeric[0])
        else:
            return None
    # extract the numeric value for the minutes and convert them to hours convert to lower case to avoid case sensitivity
    elif 'minutes' in str(time).lower() or 'minute' in str(time).lower() or 'mins' in str(time).lower() or 'min' in str(time).lower() :
        if hours_numeric:
            return int(hours_numeric[0])/60
        else:
            return None

    # Some of the data is written in a range format, so we need to calculate the average
    elif '-' in str(time) or 'to' in str(time).lower() or 'or' in str(time).lower() or '/' in str(time):
        # we calculate the average
        if len(hours_numeric) == 2:
            return (int(hours_numeric[0]) + int(hours_numeric[1]))/2
        else:
            return None
    # if the data is in a numeric format, we just return the number
    elif hours_numeric:
        return int(hours_numeric[0])
    else:
        return None

data['Study stand'] = data['Study time'].apply(standarise_time)
#data['Social stand'] = data['Time on social media'].apply(standarise_time)
#data_test = data[['Study time', 'Study stand', 'Time on social media', 'Social stand']]
#print(data_test.to_string())
#print (data['Social stand'].unique())

## print all the unique values of the column to see how to standardize them
print("----------")
print(data['Family income'].unique())



    

## Estandarizacion de Attendance

En el siguiente bloque de codigo hemos implementado la funcion standarise_attendance(value) convirtiendo los datos para que siguan el mismo formato compuesto por un numero y ya esta.

In [93]:
def standarise_attendance(attendance):
    
    # primero quitamos todos los caracteres que no sean alfanuméricos o un signo de porcentaje
    value = re.sub(r'[^a-zA-Z0-9%]', '', str(attendance))
    
    if '%' in value:
        value = re.search(r'\d+', value).group()
        value = int(value)
    # si no es un porcentaje intentar convertirlo a un número
    else:
        try:
            value = int(value)
        except ValueError:
            value = float('nan')
            
    # asegurarse de que el valor esté entre 0 y 100
    value = min(max(value, 0), 100)
    return value

data['Attendance stand'] = data['Average attendance'].apply(standarise_attendance)

print(data['Attendance stand'].unique())


[100.  90.  95.  nan  96.  85.  97.  99.  70.  80.  98.  50.  25.  87.
  40.  65.  75.   2.  60.  92.  88.   0.  10.  37.  93.  78.  57.  86.
  54.  77.  68.  45.  46.  51.  34.  56.  76.  63.  89.  79.  35.  41.
  83.  84.  67.  72.  42.  47.  23.  55.  32.  66.  30.  43.  71.  31.
  64.  73.  24.  91.  53.]


## Estandarizacion de CGPA y SGPA

En este caso en vez de definir una funcion como todos los datos que estan en formato numerico son correctos usamos to_numeric() de pandas, cuando se encuentre con una valor que no se puede convertir a numerico devolvemos NaN con errors='coerce'

In [116]:
data["Current CPGA stand"] = pd.to_numeric(data["Current CPGA"], errors='coerce')
data["Previous SGPA stand"] = pd.to_numeric(data["Previous SGPA"], errors='coerce')

print(data["Current CPGA stand"].unique())
print(data["Previous SGPA stand"].unique())

[3.64    3.53    3.89    3.5     3.65    3.81    3.82    3.48    3.32
 3.88    4.      3.9     3.8     3.52    3.67    2.74    2.8     3.1
 3.58    2.69    3.77    2.53    2.66    2.6     2.91    2.99    2.92
 2.96    2.9     3.08    2.68    2.42    3.23    2.22    3.24    2.76
 3.3     2.4     2.75    2.31    2.51        nan 3.92    3.25    3.73
 2.      3.56    3.02    3.2     3.26    3.68    3.72    2.84    2.67
 3.6     3.35    3.42    0.      3.59    2.54    3.05    2.2     3.22
 3.78    3.75    3.39    3.14    2.15    3.33    2.64    3.43    3.83
 2.98    3.79    2.89    3.18    3.03    4.67    3.7     3.4     2.95
 2.72    2.56    2.73    3.07    3.      3.38    3.21    3.45    3.74
 3.36    3.01    3.93    3.55    2.44    3.15    3.27    3.19    3.37
 2.88    2.43    3.44    3.09    3.95    3.84    3.94    3.86    3.71
 3.99    3.61    2.94    2.97    2.61    3.91    2.55    3.62    3.97
 2.1     2.25    2.01    3.295   3.69    3.29    3.41    0.13125 2.81
 2.08    3.66    2.93